In [154]:
import PyPDF2
import spacy
from spacy.matcher import Matcher
import re
import stanza

In [147]:
file_path = 'oldham.txt'
file = open(file_path, 'r')

# Read the entire file content
text = file.read()

file.close()

In [151]:
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Process the text using spaCy
doc = nlp(text)

# Keywords to search for
keywords = ["initial rent", "annual rent", "rent"]

In [129]:
keywords_match = Matcher(nlp.vocab)

match_after = Matcher(nlp.vocab)

match_before = Matcher(nlp.vocab)

for keyword in keywords:
    if len(keyword.split()) == 1:
        keywords_match.add(keyword,[
            [{"LOWER": keyword.lower()}]
        ])
        
        match_after.add(keyword + "_after",[
            [{"LOWER": keyword.lower()},
             {"OP": "*"},
             {"LOWER": {"IN": ["£", "$", "€"]}},
             {"TEXT": {"REGEX": r"[\d,.]+"}}]
        ])
        
        match_before.add(keyword + "_before",[
            [{"LOWER": {"IN": ["£", "$", "€"]}},
             {"TEXT": {"REGEX": r"[\d,.]+"}},
             {"OP": "*"},
             {"LOWER": keyword.lower()}]
        ])
    else:
        keywords_match.add(keyword,[
            [{"LOWER": keyword.split()[0].lower()},
             {"LOWER": keyword.split()[1].lower()}]
        ])
        match_after.add(keyword + "_after",[
            [{"LOWER": keyword.split()[0].lower()},
             {"LOWER": keyword.split()[1].lower()},
             {"OP": "*"},
             {"LOWER": {"IN": ["£", "$", "€"]}},
             {"TEXT": {"REGEX": r"[\d,.]+"}}]
        ])
        
        match_before.add(keyword + "_before",[
            [{"LOWER": {"IN": ["£", "$", "€"]}},
             {"TEXT": {"REGEX": r"[\d,.]+"}},
             {"OP": "*"},
             {"LOWER": keyword.split()[0].lower()},
             {"LOWER": keyword.split()[1].lower()}]
        ])
        
keywords_matched = keywords_match(doc)

rent = None

def second_match(match_func, doc, start, end):
    rent_value = None
    span = doc[start: end]
    
    rent_match = match_func(span)
    if rent_match:
        rent_value = re.search(r'[£$€][\d,.]+', span.text).group()
    return rent_value

for match_id, start, end in keywords_matched:
    search_start = max(0, start - 50)
    search_end = min(end + 100, len(doc))

    if second_match(match_after, doc, start, search_end):
        rent = second_match(match_after, doc, start, search_end)
    else:
        rent = second_match(match_before, doc, search_start, end)
        
    if rent:
        print(rent)
        break

if not rent:
    print("No rent founded.")
        


£244,580.00


In [140]:
term_keywords = "term"
term_match = Matcher(nlp.vocab)
term_val_match = Matcher(nlp.vocab)

term_match.add(term_keywords,[
    [{"LOWER": term_keywords.lower()}]
])

term_val_match.add(term_keywords+"_val",[
    [{"LOWER": term_keywords.lower()},
     {"IS_SPACE": True, "OP":"*"},
     {"TEXT": ":"},
     {"OP": "*"},
     {"LIKE_NUM": True, "OP":"+"},
     {"IS_PUNCT": True, "OP":"?"},
     {"LIKE_NUM": True, "OP":"?"},
     {"IS_PUNCT": True, "OP":"?"},
     {"IS_SPACE": True, "OP":"*"},
     {"LOWER": {"REGEX": "years?|months?"}}]
])

def second_match_2(match_func, doc, start, end):
    rent_value = None
    span = doc[start: end]
    
    rent_match = match_func(span)
    
    if rent_match:
        for match_id, start, end in rent_match:
            print(span[start:end])
#     if rent_match:
#         rent_value = re.search(r'[£$€][\d,.]+', span.text).group()
#     return rent_value

term_matched = term_match(doc)

term = None

for match_id, start, end in term_matched:
#     search_start = max(0, start - 50)
    search_end = min(end + 15, len(doc))
    span = doc[start:search_end]
    
    second_match_2(term_val_match, doc, start, search_end)
    
#     term= re.search(r'[£$€][\d,.]+', span.text).group()

#     if second_match(match_after, doc, start, search_end):
#         rent = second_match(match_after, doc, start, search_end)
#     else:
#         rent = second_match(match_before, doc, search_start, end)
        
#     if rent:
#         print(rent)
#         break

Term  : Ten (10)  years


AttributeError: 'NoneType' object has no attribute 'group'